In [1]:
import pandas as pd
import polars as pl
import geopandas as gpd
from geodatasets import get_path
import json
import pyproj

In [2]:
# nyc = gpd.read_file(get_path('nybb'))

In [3]:
# nyc.columns = ['code', 'borough', 'Shape_Leng', 'Shape_Area', 'geometry']

In [4]:
# nyc.to_parquet("data/nyc.parquet")

In [5]:
nyc = gpd.read_parquet("data/nyc.parquet")
nyc

,code,borough,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330470.010332,1.623820e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227...."
1,4,Queens,896344.047763,3.045213e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957..."
2,3,Brooklyn,741080.523166,1.937479e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."
3,1,Manhattan,359299.096471,6.364715e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940...."
4,2,Bronx,464392.991824,1.186925e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278..."


In [6]:
df = pd.read_parquet("data/borough.parquet")
df

,borough,number_of_persons_injured,number_of_persons_killed,number_of_casualty,count
0,BRONX,72466,291,72757,216786
1,BROOKLYN,157539,648,158187,466928
2,MANHATTAN,70563,351,70914,326658
3,QUEENS,120793,545,121338,392867
4,STATEN ISLAND,17809,97,17906,61409


In [7]:
nyc['lower'] = nyc['borough'].str.lower()
nyc

,code,borough,Shape_Leng,Shape_Area,geometry,lower
0,5,Staten Island,330470.010332,1.623820e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227....",staten island
1,4,Queens,896344.047763,3.045213e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957...",queens
2,3,Brooklyn,741080.523166,1.937479e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100...",brooklyn
3,1,Manhattan,359299.096471,6.364715e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940....",manhattan
4,2,Bronx,464392.991824,1.186925e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278...",bronx


In [8]:
df['lower'] = df['borough'].str.lower()
df

,borough,number_of_persons_injured,number_of_persons_killed,number_of_casualty,count,lower
0,BRONX,72466,291,72757,216786,bronx
1,BROOKLYN,157539,648,158187,466928,brooklyn
2,MANHATTAN,70563,351,70914,326658,manhattan
3,QUEENS,120793,545,121338,392867,queens
4,STATEN ISLAND,17809,97,17906,61409,staten island


In [9]:
nyc.dtypes, df.dtypes

(code             int32
 borough         object
 Shape_Leng     float64
 Shape_Area     float64
 geometry      geometry
 lower           object
 dtype: object,
 borough                      object
 number_of_persons_injured     int64
 number_of_persons_killed      int64
 number_of_casualty            int64
 count                        uint32
 lower                        object
 dtype: object)

In [10]:
merged = nyc.drop("borough", axis=1).merge(df.drop("borough", axis=1), on='lower')
merged = merged.rename(columns={"lower": "borough", "Shape_Leng": "length", "Shape_Area": "area"})
merged

,code,length,area,geometry,borough,number_of_persons_injured,number_of_persons_killed,number_of_casualty,count
0,5,330470.010332,1.623820e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227....",staten island,17809,97,17906,61409
1,4,896344.047763,3.045213e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957...",queens,120793,545,121338,392867
2,3,741080.523166,1.937479e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100...",brooklyn,157539,648,158187,466928
3,1,359299.096471,6.364715e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940....",manhattan,70563,351,70914,326658
4,2,464392.991824,1.186925e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278...",bronx,72466,291,72757,216786


In [11]:
merged = merged.to_crs(pyproj.CRS.from_epsg(4326))

In [12]:
merged.head()

,code,length,area,geometry,borough,number_of_persons_injured,number_of_persons_killed,number_of_casualty,count
0,5,330470.010332,1.623820e+09,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ...",staten island,17809,97,17906,61409
1,4,896344.047763,3.045213e+09,"MULTIPOLYGON (((-73.83668 40.59495, -73.83678 ...",queens,120793,545,121338,392867
2,3,741080.523166,1.937479e+09,"MULTIPOLYGON (((-73.86706 40.58209, -73.86769 ...",brooklyn,157539,648,158187,466928
3,1,359299.096471,6.364715e+08,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ...",manhattan,70563,351,70914,326658
4,2,464392.991824,1.186925e+09,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ...",bronx,72466,291,72757,216786


In [13]:
merged.to_parquet("data/heatmap.parquet")
# merged.to_csv("data/heatmap.csv")
# with open("data/heatmap.json", mode="w") as f:
#     json.dump(merged.to_json(), f)

In [14]:
# merged.plot(column="count",cmap='Oranges', legend=True)

In [15]:
# merged.to_file("data/heatmap.geojson", driver="GeoJSON") 

In [16]:
gdf = gpd.read_parquet("data/heatmap.parquet")
gdf

,code,length,area,geometry,borough,number_of_persons_injured,number_of_persons_killed,number_of_casualty,count
0,5,330470.010332,1.623820e+09,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ...",staten island,17809,97,17906,61409
1,4,896344.047763,3.045213e+09,"MULTIPOLYGON (((-73.83668 40.59495, -73.83678 ...",queens,120793,545,121338,392867
2,3,741080.523166,1.937479e+09,"MULTIPOLYGON (((-73.86706 40.58209, -73.86769 ...",brooklyn,157539,648,158187,466928
3,1,359299.096471,6.364715e+08,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ...",manhattan,70563,351,70914,326658
4,2,464392.991824,1.186925e+09,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ...",bronx,72466,291,72757,216786
